# Imports

In [10]:
import numpy as np
import torchvision
import time
import os
import copy
import pdb
import time
import argparse
import dask
import sys
import cv2

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms

PROJ_ROOT="/gpfs/home/s4s004/vanstee/2020-11-wisc"
sys.path.append(os.path.abspath(PROJ_ROOT + "/pytorch-retinanet"))

import retinanet.model as model

from retinanet.dataloader import CocoDataset, CSVDataset, collater, Resizer, AspectRatioBasedSampler, Augmenter, \
	UnNormalizer, Normalizer

assert torch.__version__.split('.')[0] == '1'

def nprint(mystring) :
    print("**{}** : {}".format(sys._getframe(1).f_code.co_name,mystring))

print('CUDA available: {}'.format(torch.cuda.is_available()))
# utility print function

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
print('CUDA available: {}  Using device {}'.format(torch.cuda.is_available(), os.environ['CUDA_VISIBLE_DEVICES']))
!nvidia-smi


CUDA available: True
CUDA available: True  Using device 1
Mon Nov 23 13:08:32 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 450.36.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000004:04:00.0 Off |                    0 |
| N/A   37C    P0    39W / 300W |      2MiB / 32510MiB |      0%   E. Process |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On 

# Project Configuration

In [18]:
def get_config(cfg_in={}):
    config = {}
    # Tx = 10          # variable per batch sequence length
    config["model"] = None
    config["video_file"] = "/gpfs/home/s4s004/vanstee/2020-11-wisc/inference/TownCentreXVID.avi"  # 
    config["output_directory"] = "/gpfs/home/s4s004/vanstee/2020-11-wisc/inference/TownCentre/"  # 
    config["total_frames"] = 10  # 
    config["sample_rate"] = 1  # not implemented yet

    # overwrite configs if passed
    for (k,v) in cfg_in.items() :
        if(k in config.keys()) :
            npt("Overriding Config {}:{} with {}".format(k,config[k],v))
        else :
            npt("Warning, adding a new unknown config {}:{}".format(k,v))

    # inplace update
    config.update(cfg_in)
    # npt("{}".format(config))
    
    return config

In [19]:
cfg = get_config()
nprint("{}".format(cfg))

**<module>** : {'model': None, 'video_file': '/gpfs/home/s4s004/vanstee/2020-11-wisc/inference/TownCentreXVID.avi', 'output_directory': '/gpfs/home/s4s004/vanstee/2020-11-wisc/inference/TownCentre/', 'total_frames': 10, 'sample_rate': 1}


# split_video

In [20]:
def split_video(video_file, output_directory, frame_limit=5, sample_rate=1) :

    cap  = cv2.VideoCapture(video_file)
    total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    fps = cap.get(cv2.CAP_PROP_FPS) # fps = video.get(cv2.CAP_PROP_FPS)
    secs = total_frames / fps
    nprint("Total number of frames  = {} (frames)".format(total_frames))
    nprint("Frame rate              = {} (fps)".format(fps))
    nprint("Total seconds for video = {} (s)".format(secs))
    
    
    if(frame_limit > total_frames) :
        frame_limit = int(total_frames)
    sample_rate = 1
    framecnt = 1 # equals one b/c I read a frame ...
    nprint("Total Frames to annotate= {}".format(int(frame_limit/sample_rate)))
    
    labels_string = ""
    for i in range(frame_limit) :
        if(i % 1 == 0 ):
            nprint("Loaded {} frames".format(i))
        ret, frame = cap.read()
        output_fn = output_directory + "{:0004d}".format(i)+"_stitch.jpg"
        cv2.imwrite(output_fn, frame )
        labels_string += "{},,,,,\n".format(output_fn)
    
    cap.release()
    #print(labels_string)
    f = open(output_directory + "/labels.csv", 'w')
    f.write(labels_string)
    nprint("Wrote labels to {}".format(output_directory + "/labels.csv"))
    nprint("Processing complete")

## Breaks up video into a directory with images

In [21]:
split_video(cfg["video_file"], cfg["output_directory"], frame_limit=cfg["total_frames"], sample_rate=cfg["sample_rate"]) 

**split_video** : Total number of frames  = 7500.0 (frames)
**split_video** : Frame rate              = 25.0 (fps)
**split_video** : Total seconds for video = 300.0 (s)
**split_video** : Total Frames to annotate= 10
**split_video** : Loaded 0 frames
**split_video** : Loaded 1 frames
**split_video** : Loaded 2 frames
**split_video** : Loaded 3 frames
**split_video** : Loaded 4 frames
**split_video** : Loaded 5 frames
**split_video** : Loaded 6 frames
**split_video** : Loaded 7 frames
**split_video** : Loaded 8 frames
**split_video** : Loaded 9 frames
**split_video** : Wrote labels to /gpfs/home/s4s004/vanstee/2020-11-wisc/inference/TownCentre//labels.csv
**split_video** : Processing complete


In [22]:
print(cfg["output_directory"])
print(cfg["output_directory"])
!ls {cfg["output_directory"]} | wc

/gpfs/home/s4s004/vanstee/2020-11-wisc/inference/TownCentre/
/gpfs/home/s4s004/vanstee/2020-11-wisc/inference/TownCentre/
   7502    7502  120022


# Annotate Video

In [23]:
def draw_caption(image, box, caption):

    b = np.array(box).astype(int)
    cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0), 2)
    cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)



In [24]:
def annotate_video(train_file, class_list, model, output_directory, display_image=False) :

    dataset_val = CSVDataset(train_file=train_file, class_list=class_list, 
                             transform=transforms.Compose([Normalizer(), Resizer()]))
    
    sampler_val = AspectRatioBasedSampler(dataset_val, batch_size=1, drop_last=False,)
    dataloader_val = DataLoader(dataset_val, num_workers=100, collate_fn=collater) # , batch_sampler=sampler_val
    
    retinanet = torch.load(model)
    
    use_gpu = True
    
    if use_gpu:
        if torch.cuda.is_available():
            retinanet = retinanet.cuda()
    
    if torch.cuda.is_available():
        retinanet = torch.nn.DataParallel(retinanet).cuda()
    else:
        retinanet = torch.nn.DataParallel(retinanet)
    
    retinanet.eval()
    unnormalize = UnNormalizer()
    for idx, data in enumerate(dataloader_val):
        output_filename = output_directory + "/annotated_" + "{:04d}".format(idx)+".jpg"
    
        with torch.no_grad():
            st = time.time()
            if torch.cuda.is_available():
                scores, classification, transformed_anchors = retinanet(data['img'].cuda().float())
            else:
                scores, classification, transformed_anchors = retinanet(data['img'].float())
            print('Elapsed time: {}'.format(time.time()-st))
            idxs = np.where(scores.cpu()>0.5)
            img = np.array(255 * unnormalize(data['img'][0, :, :, :])).copy()
    
            img[img<0] = 0
            img[img>255] = 255
    
            img = np.transpose(img, (1, 2, 0))
    
            img = cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_BGR2RGB)
            nprint("Detected {} boxes".format(idxs[0].shape[0]))
            for j in range(idxs[0].shape[0]):
                bbox = transformed_anchors[idxs[0][j], :]
                x1 = int(bbox[0])
                y1 = int(bbox[1])
                x2 = int(bbox[2])
                y2 = int(bbox[3])
                label_name = dataset_val.labels[int(classification[idxs[0][j]])]
                draw_caption(img, (x1, y1, x2, y2), label_name)
                cv2.rectangle(img, (x1, y1), (x2, y2), color=(0, 0, 255), thickness=2)
                
    
            # cv2.imshow('img', img)
            # cv2.waitKey(0)
            if(display_image) :
                plt.imshow(img, cmap = 'gray', interpolation = 'bicubic')
                plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
                plt.show()
            nprint("Writing File {}".format(output_filename))
            cv2.imwrite(output_filename, img )
    
    
    #annotate_images(retinanet, dataset_val, dataloader_val, output_directory, display_image=False)


# train_file = "/gpfs/home/s4s004/vanstee/2020-11-wisc/SocialDistance.convert/objdet/labels.csv"
# class_list = "/gpfs/home/s4s004/vanstee/2020-11-wisc/SocialDistance.convert/objdet/lookup.csv"


## Full Town Centre Video

In [ ]:
# Add to project config
cfg["train_file"] = "/gpfs/home/s4s004/vanstee/2020-11-wisc/inference/TownCentre/labels.csv"
cfg["class_list"] = "/gpfs/home/s4s004/vanstee/2020-11-wisc/inference/TownCentre/lookup.csv"
#cfg["model_dir"]  = "/gpfs/home/s4s004/vanstee/2020-11-wisc/training_results/model_final.pt"
cfg["model_dir"]  = "/gpfs/home/s4s004/vanstee/2020-11-wisc/csv_retinanet_45.pt"
cfg["model"]      = torch.load(cfg["model_dir"])
cfg["annot_dir"]  = "/gpfs/home/s4s004/vanstee/2020-11-wisc/inference/TownCentreAnnotated/"

annotate_video(cfg["train_file"], cfg["class_list"], cfg["model_dir"], cfg["annot_dir"], display_image=False)

# Assemble Video with Bounding Boxes

In [20]:
import glob
def assemble_video(output_directory, output_filename, debug=False) :
    output_directory
    #out = cv2.VideoWriter('project.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
    fps=25
    h=608
    w=1056
    
    output_filename = output_directory + output_filename
    print("Writing output video to {}".format(output_filename))
    out  = cv2.VideoWriter(output_filename, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w,h), True)

    filelist =  glob.glob(output_directory + '/*.jpg')
    for i in range(len(filelist)) :
        file = output_directory + "annotated_{:04d}.jpg".format(i)
        if debug : print(file)
        img = cv2.imread(file)
        height, width, layers = img.shape
        size = (width,height)
        if debug : print(size)
        out.write(img)
        #img_array.append(img)
    out.release()
 



In [ ]:
output_directory = "/gpfs/home/s4s004/vanstee/2020-11-wisc/inference/TownCentreAnnotated/"
assemble_video(output_directory, "example.mp4")

# Covid Social Distancing Example

General algorithm, 
1. for each detected person box, calculate foot pixel position
2. then use homographic projection to map to real world coordinates
3. then use simple KD tree to determine collision
4. render collisions a different color 

## Camera Calibration


In [13]:
import numpy as np  # import the numpy library
											

# the following two parallel arrays allow us to map from a source of image coordinates in a tilted plane
# to a target set of corrdinates in a destination frame
# use a proam such as ImageMouseClick.py
# to create a list of coordinates for points where you know or can calulate the world coordinates
# for example image coordinates [143, 468] was found to be associated with  world coordinate of 10',25'

pts_src = np.array([[143, 468], [789,178], [541,544], [1194, 671], [1083,214], [1560, 273]])
pts_dst = np.array([[10,25],    [10,65],   [24.7,25], [44.7,25],   [24.7, 65], [44.7,65]])

# calculate matrix H. DO this once for an image to compute H for camera setup, 
# resuse H over and over for rest of images

homograpy, status  = cv2.findHomography(pts_src, pts_dst)
homograpy_inv, status = cv2.findHomography(pts_dst, pts_src)

In [14]:
# Verify .. Convert Px space to World Coordinates
# footPositionPx = np.array([[[1250,250], [989,893]]], dtype='float32')
footPositionPx = np.array([[[143, 468], [789,178], [541,544], [1194, 671], [1083,214], [1560, 273]]], dtype='float32')
print(footPositionPx.shape)
FootPostionWorldWc = cv2.perspectiveTransform(footPositionPx, homograpy)
FootPostionWorldWc

# FootPostionWorldWc = cv2.perspectiveTransform(footPositionPx, h)




(1, 6, 2)


array([[[10.013301, 24.920633],
        [10.161524, 65.04503 ],
        [24.677074, 25.029789],
        [44.709366, 25.050034],
        [24.425364, 65.02927 ],
        [44.813377, 64.925255]]], dtype=float32)

In [15]:
#x1,y1,x2,y2 (upper left -> lower right)
from scipy.spatial import KDTree

def calc_foot_position(bbox) : 
        x1 = int(bbox[0])
        y1 = int(bbox[1])
        x2 = int(bbox[2])
        y2 = int(bbox[3])    
        return (int((x1+x2)/2), int(y2))

def get_pairs(point_list_px,dist=6) :
    #print(point_list_px)
    # Transform to real world coordinates
    point_list_px = np.expand_dims(point_list_px, axis=0)
    point_list_px = point_list_px.astype('float32')
    
    #print(point_list_px.shape)
    point_list_wc = cv2.perspectiveTransform(point_list_px, homograpy)
    point_list_wc = point_list_wc.squeeze(0)
    # T = KDTree(FootPostionWorldXY[0,:])
    tree = KDTree(point_list_wc)
    pairs = list(tree.query_pairs(r=dist))
    #print("Found pairs {}".format(pairs))
    # Create a simple dictionary to highlight index that has collsions
    box_collision_dict  = {}
    for p in pairs :
        box_collision_dict[p[0]] = 1
        box_collision_dict[p[1]] = 1
    #print(box_collision_dict)
    return box_collision_dict

def draw_covid_boxes(img, idxs,classification, transformed_anchors, label_names) :
    
    foot_position_px_list = []
    for j in range(idxs[0].shape[0]):
        bbox = transformed_anchors[idxs[0][j], :]
        foot_position_px_list.append(calc_foot_position(bbox))
    
    box_collision_dict = get_pairs(np.array(foot_position_px_list))

    for j in range(idxs[0].shape[0]):
        bbox = transformed_anchors[idxs[0][j], :]
        x1 = int(bbox[0])
        y1 = int(bbox[1])
        x2 = int(bbox[2])
        y2 = int(bbox[3])
        foot_position_px = calc_foot_position(bbox)
        
        label_name = label_names[int(classification[idxs[0][j]])]
        draw_caption(img, (x1, y1, x2, y2), label_name)
        #print(j, box_collision_dict.keys())
        if( j in box_collision_dict.keys()) :
            # collision
            cv2.rectangle(img, (x1, y1), (x2, y2), color=(0, 0, 255), thickness=2) #BGR !
        else :
            cv2.rectangle(img, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2)

        # Create an overlay for a warped circle ..
        
        overlay = np.zeros((img.shape[0],img.shape[1]), np.uint8)
        cv2.circle(overlay, foot_position_px, 50, 255)
        cp = np.transpose(np.where(img==255))
        #cv2.addWeighted(overlay, 0.75, output, 1 - alpha, 0, output)

In [16]:
# Test
point_list = np.array([[1,2],[1,1],[10,10]])
get_pairs(point_list)

{0: 1, 1: 1, 2: 1}

In [17]:
def draw_simple_boxes(img, idxs,classification, transformed_anchors, label_names) :
    for j in range(idxs[0].shape[0]):
        bbox = transformed_anchors[idxs[0][j], :]
        #print(bbox)
        x1 = int(bbox[0])
        y1 = int(bbox[1])
        x2 = int(bbox[2])
        y2 = int(bbox[3])
        label_name = label_names[int(classification[idxs[0][j]])]
        draw_caption(img, (x1, y1, x2, y2), label_name)
        cv2.rectangle(img, (x1, y1), (x2, y2), color=(0, 0, 255), thickness=2)



In [28]:
def annotate_video_covid(cfg, display_image=False, debug=False) :

    dataset_val = CSVDataset(train_file=cfg["train_file"], class_list=cfg["class_list"], 
                             transform=transforms.Compose([Normalizer(), Resizer()]))
    
    # sampler_val = AspectRatioBasedSampler(dataset_val, batch_size=1, drop_last=False,)
    dataloader_val = DataLoader(dataset_val, num_workers=100, collate_fn=collater, batch_size=1) # , batch_sampler=sampler_val
    
    retinanet = torch.load(cfg["model_dir"])
    
    use_gpu = True
    
    if use_gpu:
        if torch.cuda.is_available():
            retinanet = retinanet.cuda()
    
    if torch.cuda.is_available():
        retinanet = torch.nn.DataParallel(retinanet).cuda()
    else:
        retinanet = torch.nn.DataParallel(retinanet)
    
    retinanet.eval()
    unnormalize = UnNormalizer()
    for idx, data in enumerate(dataloader_val):
        output_filename = cfg["annot_dir"] + "/annotated_" + "{:04d}".format(idx)+".jpg"
    
        with torch.no_grad():
            st = time.time()
            if torch.cuda.is_available():
                scores, classification, transformed_anchors = retinanet(data['img'].cuda().float())
            else:
                scores, classification, transformed_anchors = retinanet(data['img'].float())
            if debug : print('Elapsed time: {}'.format(time.time()-st))
            idxs = np.where(scores.cpu()>0.5)
            img = np.array(255 * unnormalize(data['img'][0, :, :, :])).copy()
    
            img[img<0] = 0
            img[img>255] = 255
    
            img = np.transpose(img, (1, 2, 0))
    
            img = cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_BGR2RGB)
            if debug : nprint("Detected {} boxes".format(idxs[0].shape[0]))            
            #draw_simple_boxes(img, idxs,classification, transformed_anchors, dataset_val.labels)
            draw_covid_boxes(img, idxs,classification, transformed_anchors, dataset_val.labels)
    
            # cv2.imshow('img', img)
            # cv2.waitKey(0)
            if(display_image) :
                plt.figure(figsize = (15,15))
                plt.imshow(img, interpolation = 'bicubic')
                plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
                plt.show()
            if debug : nprint("Writing File {}".format(output_filename))
            cv2.imwrite(output_filename, img )


In [29]:
cfg["train_file"] = "/gpfs/home/s4s004/vanstee/2020-11-wisc/inference/TownCentre/labels.csv"
cfg["class_list"] = "/gpfs/home/s4s004/vanstee/2020-11-wisc/inference/TownCentre/lookup.csv"
cfg["model_dir"]  = "/gpfs/home/s4s004/vanstee/2020-11-wisc/training_results/model_final.pt"
cfg["model"]      = torch.load(cfg["model_dir"])
cfg["annot_dir"]  = "/gpfs/home/s4s004/vanstee/2020-11-wisc/inference/TownCentreCovid/"
annotate_video_covid(cfg, display_image=False) 

KeyboardInterrupt: 

# Assemble Video for Social Distancing

In [ ]:
output_directory = "/gpfs/home/s4s004/vanstee/2020-11-wisc/inference/TownCentreCovid/"
assemble_video(output_directory, "example.mp4")

# Visualize in world coordinates

In [31]:
def plot_combo(onefile, rnd_base = 0):
    # homagraphy transform of image points
    
    repeatfile = onefile

    imageIn = np.zeros([1,len(repeatfile),2], np.float32)
    imageIn[0,:,0] = (repeatfile.xmin + repeatfile.xmax)/2
    imageIn[0,:,1] = repeatfile.ymax
    #imageIn = list(zip(onefile.xmin, onefile.ymin))
    pointsOut = cv2.perspectiveTransform(imageIn, h)

    streetLineDots = [[11, 884], [1086, 214], [39, 910], 
                   [1076, 243], [914, 905], [1511, 281], 
                   [975, 915], [1488, 345], 
                   [978, 911], [1482, 355]]
    streetLineDotsIn = np.zeros([1,len(streetLineDots),2], np.float32)
    streetLineDotsIn[0,:] = streetLineDots
    streetLineDotsOut = cv2.perspectiveTransform(streetLineDotsIn, h)

    fig  = plt.subplots(figsize=(18,9))
    gs = gridspec.GridSpec(1, 2, width_ratios=[1,4]) 

    # plot chart on left  ***************************************************************************
    ax0 = plt.subplot(gs[0])
    ylmin = 0 #int(min(pointsOut[0,:,1]))
    ylmax = 100 # 1 + int(max(pointsOut[0,:,1]+1))
    
    # streets *******************************************
    rect = plt.Rectangle((26, ylmin), 20, ylmax, color='k', alpha=0.1)
    ax0.add_patch(rect)
    rect = plt.Rectangle((27, ylmin), 18, ylmax, color='w', alpha=0.4)
    ax0.add_patch(rect)
    
    # sidewalk *******************************************
    rect = plt.Rectangle((10, 66), 16, 9, color='k', alpha=0.1)
    ax0.add_patch(rect)
    
    
    # ******************************************************
    # plot posiotn on strip chart to left - green
    # plot center blue point
    if rnd_base == 0 :
        xrnd = pointsOut[0,:,0]
        yrnd = pointsOut[0,:,1]
    else:
        xrnd = rnd_base * np.round(pointsOut[0,:,0]/rnd_base)
        yrnd = rnd_base * np.round(pointsOut[0,:,1]/rnd_base)
    plt.scatter(xrnd, yrnd, alpha = 1)
    # plot green bubble
    plt.scatter(xrnd, yrnd, s=720, facecolors='none', edgecolors='lime', linewidths = 3)
    #plt.scatter(streetLineDotsOut[0,:,0], streetLineDotsOut[0,:,1], c='k')
    plt.xticks(rotation = -90)
    ax0.set_xticks(np.arange(0,60,3))
    ax0.set_yticks(np.arange(0, 100,3))
    plt.ylim(0, 100)
    plt.xlim(0, 60)
    plt.grid(c='cyan')

    pers = np.zeros([1,len(repeatfile),2], np.float32)
    pers[0,:,0] = (repeatfile.xmin + repeatfile.xmax)/2
    pers[0,:,1] = repeatfile.ymax
    #a = list(zip(onefile.xmin, onefile.ymin))
    pointsOut = cv2.perspectiveTransform(pers, h)

    T = KDTree(pointsOut[0,:])
    #test = pointsOut[0,0]
    #idx = T.query_ball_point(test,r=10)
    #pointsOut[0,idx]
    # ***********************************
    # ************************************
    
    R = 7
    
    
    # ***********************************
    # ************************************
    # plot posiotn on strip chart to left, pair of people  - red
    pairs = T.query_pairs(r=R)
    idx = 0
    d = Dmetric(pointsOut, T.query_pairs(R))
    for pair in pairs:
        #left pair
        if rnd_base == 0 :
            xrnd = pointsOut[0,pair[0],0]
            yrnd = pointsOut[0,pair[0],1]
        else:
            xrnd = rnd_base * np.round(pointsOut[0,pair[0],0]/rnd_base)
            yrnd = rnd_base * np.round(pointsOut[0,pair[0],1]/rnd_base) 
        plt.scatter(xrnd, yrnd, s=720, facecolors='none', edgecolors='r', linewidths = 3)
        #right pair
        if rnd_base == 0 :
            xrnd = pointsOut[0,pair[1],0]
            yrnd = pointsOut[0,pair[1],1]
        else:
            xrnd = rnd_base * np.round(pointsOut[0,pair[1],0]/rnd_base)
            yrnd = rnd_base * np.round(pointsOut[0,pair[1],1]/rnd_base)        
        #xrnd = base * np.round(pointsOut[0,pair[1],0]/base) 
        #yrnd = base * np.round(pointsOut[0,pair[1],1]/base)    
        plt.scatter(xrnd, yrnd, s=720, facecolors='none', edgecolors='r', linewidths = 3) 
        #ax0.text(46, 93, 'Distance\nAlert', fontsize=15,
            #bbox={'facecolor': 'red', 'alpha': 0.5, 'pad': 10})
        #d = Dmetric(pointsOut, T.query_pairs(R))
        #plt.text(pointsOut[0,pair[0],0], pointsOut[0,pair[0],1] , '{:5.1f}'.format(d[idx]), color = 'red', fontsize=15)
        idx += 1


    # plot Image on right *******************************************************************
    # ***************************************************************************************
    ax1 = plt.subplot(gs[1])
    img_fn = './Frames/{}'.format(onefile.iloc[0].image)
    img = cv2.imread(img_fn)
    img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img2, interpolation='nearest')
    # green rectangles
    for index, person in onefile.iterrows():
        # Create a Rectangle patch
        rect = patches.Rectangle((person['xmin'],person['ymin']), 
                                 person['width'],person['ymax']-person['ymin'],
                                 linewidth=1,
                                 edgecolor='lime',
                                 facecolor='none')
        # Add the patch to the Axes
        ax1.add_patch(rect)
    # RED rectangles for pairs
    T = KDTree(pointsOut[0,:])
    pairs = T.query_pairs(r=R)
    idx = 0
    d = Dmetric(pointsOut, T.query_pairs(R))
    for pair in pairs:
        one = repeatfile.iloc[pair[0]]
        widthOne = one.xmax - one.xmin
        heightOne = one.ymax - one.ymin
        xminsOne = one.xmin
        yminsOne = one.ymin
        rect = patches.Rectangle((xminsOne, yminsOne), 
                                 widthOne, heightOne,
                                 linewidth= 3,
                                 edgecolor='r',
                                 facecolor='none')
        # Add the patch to the Axes
        ax1.add_patch(rect)

        two = repeatfile.iloc[pair[1]]
        widthTwo = two.xmax - two.xmin
        heightTwo = two.ymax - two.ymin
        xminsTwo = two.xmin
        yminsTwo = two.ymin
        rect = patches.Rectangle((xminsTwo, yminsTwo), 
                                 widthTwo, heightTwo,
                                 linewidth= 3,
                                 edgecolor='r',
                                 facecolor='none')
        # Add the patch to the Axes
        ax1.add_patch(rect)    
        ax1.text(20, 80, 'Distance\nAlert < 6 feet', fontsize=15,
            bbox={'facecolor': 'red', 'alpha': 0.5, 'pad': 10})
        #d = Dmetric(pointsOut, T.query_pairs(R))
        ####plt.text(pointsOut[0,pair[0],0], pointsOut[0,pair[0],1], '{:5.1f}'.format(d[idx]), color = 'red', fontsize=15)
        #plt.text(onefile.iloc[pair[0]].xmax, onefile.iloc[pair[0]].ymax , '{:5.1f}'.format(d[idx]) , color = 'red', fontsize=15)
        idx += 1
    plt.tight_layout()  
    newOutput = '{:04d}'.format(int(onefile.iloc[0,:].image.split('-')[-1].split('.')[0]))
    plt.savefig('./Alerts/alert-{}'.format(newOutput))
    plt.clf()
    #plt.close()
    #plt.show()
    return

In [ ]:

persons_df = pd.read_csv('persons_df.csv')
#for index, ones in persons_df.iterrows():
personsList = sorted(persons_df.image.unique())
#personsList = ['out-0005.jpg']
i = 0
#lastOne = persons_df[persons_df.image == 'out-0001.jpg']
for fn in personsList:
    onefile = persons_df[persons_df.image == fn]
    plot_combo(onefile)  
    if (i % 1) == 0:
        print (i,' out of 7500')
    i += 1
    plt.clf()
print("done")